# Лабораторная работа 2, студент Устинов Денис Александрович М8О-406Б-21

## 1. Выбор начальных условий

### a. Набор данных для задачи классификации

Задача: Определение типа активности человека на основе данных с носимых устройств

Набор данных: Human Activity Recognition Dataset (HAR) (https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones), где данные получены с носимых датчиков и классифицируются в такие категории, как ходьба, сидение, подъем по лестнице и т. д.

Обоснование выбора: HAR представляет практическую задачу для классификации с несколькими классами и используется в задачах мониторинга здоровья и физической активности. Задача распознавания активности с помощью KNN полезна и практична, так как такой алгоритм достаточно прост для реализации в системах с носимыми устройствами.

### b. Набор данных для задачи регрессии

Задача: Прогнозирование уровня выбросов CO2 автомобилей

Набор данных: CO2 Emission by Vehicles (https://www.kaggle.com/datasets/debajyotipodder/co2-emission-by-vehicles), включающий параметры автомобиля (тип двигателя, вес, объем двигателя) и уровень выбросов CO2.

Обоснование выбора: Прогнозирование выбросов позволяет оценивать экологическое воздействие автомобилей на основе их характеристик, что является важной задачей в экологии и инженерии. Такой набор данных также подходит для KNN, так как близость характеристик автомобилей позволяет строить точные прогнозы выбросов.

### c. Выбор метрик качества

Для задачи классификации:

1) Accuracy (Точность): доля правильно классифицированных объектов. Хорошо подходит для задачи с равномерным распределением классов.
2) Precision: можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными
3) Recall: показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.
2) F1-score: среднее гармоническое между точностью (precision) и полнотой (recall), особенно полезно, если классы несбалансированы, так как учитывает ложноположительные и ложноотрицательные предсказания.

Для задачи регрессии:

1) Mean Absolute Error (MAE): средняя абсолютная ошибка, показывает среднее отклонение предсказаний от фактических значений и хорошо интерпретируется.
2) Mean Squared Error (MSE): средняя квадратичная ошибка, показывает более крупные ошибки больше чем MAE, что делает его полезным для выявления больших отклонений в прогнозах.

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sklearn
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier


#### Предобработаем данные

Получение и очистка данных

In [2]:
features = list()
with open('UCI HAR Dataset/features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]

Дубликаты:

In [3]:
seen = set()
uniq_features = []
for idx, x in enumerate(features):
    if x not in seen:
        uniq_features.append(x)
        seen.add(x)
    elif x + 'n' not in seen:
        uniq_features.append(x + 'n')
        seen.add(x + 'n')
    else:
        uniq_features.append(x + 'nn')
        seen.add(x + 'nn')
len(uniq_features)

561

Подготовим обучающую выборку

In [4]:
def map_func(arg):
    if arg == 1: 
        return 'WALKING'
    if arg == 2:
        return 'WALKING_UPSTAIRS'
    if arg == 3:
        return 'WALKING_DOWNSTAIRS'
    if arg == 4:
        return 'SITTING'
    if arg == 5:
        return 'STANDING'
    if arg == 6:
        return 'LAYING'

In [5]:
X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)
X_train['subject'] = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', header=None)

y_train = pd.read_csv('UCI HAR Dataset/train/y_train.txt', names=['Activity'])
y_train_labels = y_train.map(map_func)

train = X_train
train['Activity'] = y_train
train['ActivityName'] = y_train_labels
train.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3107987138.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
3312,0.280274,-0.012609,-0.1072,-0.998698,-0.984339,-0.992663,-0.99929,-0.982526,-0.992022,-0.941483,...,-0.083526,-0.139677,0.109975,-0.04667,-0.63593,0.110069,0.280759,17,4,SITTING


Подготовим тестовую выборку

In [6]:
X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)
X_test['subject'] = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', header=None)

y_test = pd.read_csv('UCI HAR Dataset/test/y_test.txt', names=['Activity'])
y_test_labels = y_test.map(map_func)

test = X_test
test['Activity'] = y_test
test['ActivityName'] = y_test_labels
test.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/2096605161.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
1075,0.237046,-0.099672,-0.057156,-0.928608,-0.11664,-0.621681,-0.932908,-0.114518,-0.615301,-0.927723,...,-0.275683,0.289203,0.089517,-0.016266,-0.368489,0.468678,-0.198403,10,4,SITTING


Поменяем названия функций

In [7]:
columns = train.columns

columns = columns.str.replace('[()]','')
columns = columns.str.replace('[-]', '')
columns = columns.str.replace('[,]','')

train.columns = columns
test.columns = columns

test.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity',
       'ActivityName'],
      dtype='object', length=564)

Сохраним выборки как csv

In [8]:
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

### a. Обучить модели логистической регрессии из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pandas as pd
import numpy as np
import sklearn.utils

def read_data(file):
    data = pd.read_csv(file)
    
    data = sklearn.utils.shuffle(data)
    
    X_data = data.drop(['subject', 'Activity', 'ActivityName'], axis=1)
    y_data = data.ActivityName
    
    return np.array(X_data), np.array(y_data)

def train_model(train_x, train_y, validation=None):
    model = LogisticRegression()
    
    model.fit(train_x, train_y)
    
    if validation is not None:
        y_hat = model.predict(validation[0])
        acc = metrics.accuracy_score(validation[1], y_hat)
        print(f"Accuracy = {acc}")
        recall = metrics.recall_score(validation[1], y_hat, average='weighted')
        precision = metrics.precision_score(validation[1], y_hat, average='weighted')
        f1 = metrics.f1_score(validation[1], y_hat, average='weighted')
        print(f"Recall = {recall}")
        print(f"Precision = {precision}")
        print(f"F1 Score = {f1}")
    
    return model

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

model = train_model(train_X, train_y, validation=(test_X, test_y))


Accuracy = 0.9586019681031558
Recall = 0.9586019681031558
Precision = 0.9597941694255984
F1 Score = 0.9584674380821284


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### b. Обучить модели из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)

y_pred = linear_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"MAE (CO2 Emissions): {mae:.2f}")
print(f"MSE (CO2 Emissions): {mse:.2f}")


MAE (CO2 Emissions): 13.13
MSE (CO2 Emissions): 377.79


## 3. Улучшение бейзлайна

### a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.

2.  Подбор гиперпараметров
    - Для логистической регрессии:

        - C (инверсия регуляризации): Найти оптимальный уровень регуляризации. Маленький C — сильная регуляризация, большой C — слабая.
        - solver: Подобрать лучший метод оптимизации (например, 'lbfgs', 'liblinear', 'saga' и т.д.) для конкретных данных.

    - Для линейной регрессии:

        - alpha: Параметр регуляризации для методов Ridge и Lasso.
        - Проверить влияние применения регуляризованных моделей:
            - Ridge (L2-регуляризация).
            - Lasso (L1-регуляризация).

3. Методы оптимизации
    - Grid Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели.

### Обучение моделей, оценка качества обучения моделей по метрикам

#### Масштабируем данные

In [11]:
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

#### Подбор гиперпараметров и Grid Search

#### Логистическая регрессия (классификация)

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

param_grid_logreg = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': [1000, 5000, 10000],
    'solver': ['lbfgs', 'liblinear'],
}

logreg = LogisticRegression(solver='lbfgs')
grid_search_logreg = GridSearchCV(logreg, param_grid_logreg, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_logreg.fit(train_X_scaled, train_y)
best_logreg = grid_search_logreg.best_estimator_

print("Лучшие параметры для логистической регрессии (HAR):", grid_search_logreg.best_params_)
print("Лучший результат на кросс-валидации (HAR):", grid_search_logreg.best_score_)

y_pred_logreg = best_logreg.predict(test_X_scaled)
accuracy = accuracy_score(test_y, y_pred_logreg)
precision = precision_score(test_y, y_pred_logreg, average='weighted')
recall = recall_score(test_y, y_pred_logreg, average='weighted')
f1 = f1_score(test_y, y_pred_logreg, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Лучшие параметры для логистической регрессии (HAR): {'C': 1, 'max_iter': 1000, 'solver': 'liblinear'}
Лучший результат на кросс-валидации (HAR): 0.9849009188991709
Accuracy: 0.9613165931455717
Precision: 0.963249239599378
Recall: 0.9613165931455717
F1 Score: 0.9613709750318309


#### Линейная регрессия (регрессия)

In [13]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

param_grid_ridge = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'saga']
}

ridge = Ridge()
grid_search_ridge = GridSearchCV(ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_ridge.fit(X_train, y_train)
best_ridge = grid_search_ridge.best_estimator_

print("Лучшие параметры для Ridge регрессии (CO2):", grid_search_ridge.best_params_)
print("Лучший результат на кросс-валидации (CO2):", -grid_search_ridge.best_score_)

y_pred_ridge = best_ridge.predict(X_test)
print("MAE (CO2 Emissions):", mean_absolute_error(y_test, y_pred_ridge))
print("MSE (CO2 Emissions):", mean_squared_error(y_test, y_pred_ridge))

param_grid_lasso = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'max_iter': [1000, 5000, 10000]
}

lasso = Lasso()
grid_search_lasso = GridSearchCV(lasso, param_grid_lasso, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_lasso.fit(X_train, y_train)
best_lasso = grid_search_lasso.best_estimator_

print("Лучшие параметры для Lasso регрессии (CO2):", grid_search_lasso.best_params_)
print("Лучший результат на кросс-валидации (CO2):", -grid_search_lasso.best_score_)

y_pred_lasso = best_lasso.predict(X_test)
print("MAE (CO2 Emissions):", mean_absolute_error(y_test, y_pred_lasso))
print("MSE (CO2 Emissions):", mean_squared_error(y_test, y_pred_lasso))


Лучшие параметры для Ridge регрессии (CO2): {'alpha': 10, 'solver': 'lsqr'}
Лучший результат на кросс-валидации (CO2): 375.35785425391526
MAE (CO2 Emissions): 13.12377206913496
MSE (CO2 Emissions): 380.5580689476625


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.332e+04, tolerance: 1.443e+03
  model = cd_fast.enet_coordinate_descent(


Лучшие параметры для Lasso регрессии (CO2): {'alpha': 0.01, 'max_iter': 1000}
Лучший результат на кросс-валидации (CO2): 375.7968494885642
MAE (CO2 Emissions): 13.146135034120853
MSE (CO2 Emissions): 378.50093303746394


#### Добавление нового признака (для задачи регрессии)

In [25]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

ridge = Ridge(max_iter=20000)
param_grid_ridge = {
    'alpha': [0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'saga']
}
grid_search_ridge = GridSearchCV(ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_ridge.fit(X_train_poly_scaled, y_train_new_field)
best_ridge = grid_search_ridge.best_estimator_

y_pred_ridge = best_ridge.predict(X_test_poly_scaled)
ridge_mae = mean_absolute_error(y_test_new_field, y_pred_ridge)
ridge_mse = mean_squared_error(y_test_new_field, y_pred_ridge)

print("Лучшие параметры для Ridge:", grid_search_ridge.best_params_)
print(f"Ridge MAE: {ridge_mae:.2f}")
print(f"Ridge MSE: {ridge_mse:.2f}")

lasso = Lasso(max_iter=20000)
param_grid_lasso = {
    'alpha': [0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': [20000]
}
grid_search_lasso = GridSearchCV(lasso, param_grid_lasso, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_lasso.fit(X_train_poly_scaled, y_train_new_field)
best_lasso = grid_search_lasso.best_estimator_

y_pred_lasso = best_lasso.predict(X_test_poly_scaled)
lasso_mae = mean_absolute_error(y_test_new_field, y_pred_lasso)
lasso_mse = mean_squared_error(y_test_new_field, y_pred_lasso)

print("Лучшие параметры для Lasso:", grid_search_lasso.best_params_)
print(f"Lasso MAE: {lasso_mae:.2f}")
print(f"Lasso MSE: {lasso_mse:.2f}")


Лучшие параметры для Ridge: {'alpha': 0.1, 'solver': 'svd'}
Ridge MAE: 7.29
Ridge MSE: 139.02


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.221e+03, tolerance: 1.421e+03
  model = cd_fast.enet_coordinate_descent(
/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.627e+03, tolerance: 1.434e+03
  model = cd_fast.enet_coordinate_descent(
/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider

Лучшие параметры для Lasso: {'alpha': 0.01, 'max_iter': 20000}
Lasso MAE: 7.45
Lasso MSE: 147.72


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.905e+03, tolerance: 1.779e+03
  model = cd_fast.enet_coordinate_descent(


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации:
{'C': 1, 'max_iter': 1000, 'solver': 'liblinear'}

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pandas as pd
import numpy as np
import sklearn.utils

def read_data(file):
    data = pd.read_csv(file)
    
    data = sklearn.utils.shuffle(data)
    
    X_data = data.drop(['subject', 'Activity', 'ActivityName'], axis=1)
    y_data = data.ActivityName
    
    return np.array(X_data), np.array(y_data)

def train_model(train_x, train_y, validation=None):
    model = LogisticRegression(max_iter=1000, solver='liblinear', multi_class='ovr', C = 1)
    
    model.fit(train_x, train_y)
    
    if validation is not None:
        y_hat = model.predict(validation[0])
        acc = metrics.accuracy_score(validation[1], y_hat)
        print(f"Accuracy = {acc}")
        recall = metrics.recall_score(validation[1], y_hat, average='weighted')
        precision = metrics.precision_score(validation[1], y_hat, average='weighted')
        f1 = metrics.f1_score(validation[1], y_hat, average='weighted')
        print(f"Recall = {recall}")
        print(f"Precision = {precision}")
        print(f"F1 Score = {f1}")
    
    return model

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

model = train_model(train_X, train_y, validation=(test_X, test_y))

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy = 0.9619952494061758
Recall = 0.9619952494061758
Precision = 0.9632384391448742
F1 Score = 0.9618652634530593


#### Задача регрессии

Более лучшие (по MAE, по кросс-валидации и по метрикам после добавления нового признака) для задачи регрессии были показаны моделью Ridge, поэтому улучшенный бейзлайн смормируем на основе модели Ridge с гиперпараметрами:
{'alpha': 0.1, 'solver': 'svd'}

In [2]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

ridge = Ridge(max_iter=20000, alpha=0.1, solver='svd')
ridge.fit(X_train_poly_scaled, y_train_new_field)

y_pred_ridge = ridge.predict(X_test_poly_scaled)
ridge_mae = mean_absolute_error(y_test_new_field, y_pred_ridge)
ridge_mse = mean_squared_error(y_test_new_field, y_pred_ridge)

print(f"MAE: {ridge_mae:.2f}")
print(f"MSE: {ridge_mse:.2f}")

MAE: 7.29
MSE: 139.02


### Выводы

В задаче классификации на наборе данных HAR модель логистической регрессии без оптимизаций продемонстрировала хорошие результаты с точностью около 96%. Однако дальнейшая оптимизация с помощью Grid Search и Random Search позволила немного улучшить результаты, достигнув точности в 96.13% и F1-меры 0.96, что подтвердило важность настройки гиперпараметров для улучшения классификации.

В задаче регрессии для прогноза выбросов CO2 модель линейной регрессии без улучшений показала значения MAE и MSE около 13.13 и 377.79 соответственно. После подбора гиперпараметров для Ridge и Lasso моделей ошибки немного снизились, но наибольшее улучшение было достигнуто при добавлении нового признака Engine_Fuel, который позволил существенно уменьшить MAE и MSE, снизив их до 7.29 и 139.02 для Ridge.

Комбинирование настройки гиперпараметров и создания новых признаков значительно улучшило результаты моделей, особенно в задаче регрессии, где добавление информативных признаков дало наибольшее улучшение. Эти подходы подтверждают свою важность для повышения качества моделей в машинном обучении.

## 4. Имплементация алгоритма машинного обучения 

### Имплементация логистической регрессии для классификации

In [6]:
import numpy as np


class LogisticRegressionCustom:
    def __init__(self, lr=0.01, max_iter=1000, penalty=None, C=1.0, tol=1e-4):
        self.lr = lr
        self.max_iter = max_iter
        self.penalty = penalty
        self.C = C
        self.tol = tol

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X_train, y_train):
        self.X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
        self.y_train = y_train
        self.weights = np.zeros(self.X_train.shape[1])
        
        for _ in range(self.max_iter):
            predictions = self._sigmoid(np.dot(self.X_train, self.weights))
            errors = predictions - self.y_train
            gradient = np.dot(self.X_train.T, errors) / len(y_train)
            
            if self.penalty == 'l2':
                gradient += (1 / self.C) * self.weights
            elif self.penalty == 'l1':
                gradient += (1 / self.C) * np.sign(self.weights)
            
            self.weights -= self.lr * gradient
            
            if np.linalg.norm(gradient) < self.tol:
                break

    def predict_proba(self, X_test):
        X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

        return self._sigmoid(np.dot(X_test, self.weights))

    def predict(self, X_test):
        probabilities = self.predict_proba(X_test)

        return (probabilities >= 0.5).astype(int)

### Имплементация линейной регрессии для регрессии

In [7]:
class LinearRegressionCustom:
    def __init__(self, lr=0.01, max_iter=1000, penalty=None, alpha=0.0, tol=1e-4):
        self.lr = lr
        self.max_iter = max_iter
        self.penalty = penalty
        self.alpha = alpha
        self.tol = tol

    def fit(self, X_train, y_train):
        self.X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
        self.y_train = y_train
        self.weights = np.zeros(self.X_train.shape[1])
        
        for _ in range(self.max_iter):
            predictions = np.dot(self.X_train, self.weights)
            errors = predictions - self.y_train
            gradient = np.dot(self.X_train.T, errors) / len(y_train)
            
            if self.penalty == 'l2':
                gradient += (2 * self.alpha) * self.weights
            elif self.penalty == 'l1':
                gradient += self.alpha * np.sign(self.weights)
            
            self.weights -= self.lr * gradient
            
            if np.linalg.norm(gradient) < self.tol:
                break

    def predict(self, X_test):
        X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))
        
        return np.dot(X_test, self.weights)

### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [17]:
def normalize_data(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

def encode_labels(y):
    label_to_num = {label: idx for idx, label in enumerate(np.unique(y))}
    num_to_label = {idx: label for label, idx in label_to_num.items()}
    y_encoded = np.array([label_to_num[label] for label in y])
    return y_encoded, label_to_num, num_to_label

def train_custom_logistic_regression(train_x, train_y, validation=None):
    train_x_normalized = normalize_data(train_x)
    train_y_encoded, _, _ = encode_labels(train_y)

    if validation is not None:
        validation_x_normalized = normalize_data(validation[0])
        validation_y_encoded, _, _ = encode_labels(validation[1])
    else:
        validation_x_normalized = None
        validation_y_encoded = None

    model = LogisticRegressionCustom(lr=0.01, max_iter=2000, penalty='l1')
    model.fit(train_x_normalized, train_y_encoded)

    y_hat = model.predict(validation_x_normalized)
    acc = metrics.accuracy_score(validation_y_encoded, y_hat)
    print(f"Accuracy = {acc}")
    recall = metrics.recall_score(validation_y_encoded, y_hat, average='weighted')
    precision = metrics.precision_score(validation_y_encoded, y_hat, average='weighted')
    f1 = metrics.f1_score(validation_y_encoded, y_hat, average='weighted')
    print(f"Recall = {recall}")
    print(f"Precision = {precision}")
    print(f"F1 Score = {f1}")

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_custom_logistic_regression(train_X, train_y, validation=(test_X, test_y))

Accuracy = 0.18900576857821513
Recall = 0.18900576857821513
Precision = 0.07599169489560975
F1 Score = 0.10811883566310325


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Задача регрессии

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values.astype(np.float64)
y = data['CO2 Emissions(g/km)'].values.astype(np.float64)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

custom_linear_regressor = LinearRegressionCustom(lr=0.01, max_iter=1000)
custom_linear_regressor.fit(X_train, y_train)

predictions = custom_linear_regressor.predict(X_test)

mae = np.mean(np.abs(predictions - y_test))
mse = np.mean((predictions - y_test) ** 2)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")


MAE: 13.18
MSE: 384.77


### Сравнение результатов с п.2. Выводы

Результаты сравнения моделей показывают, что самописные реализации логистической и линейной регрессий уступают по качеству моделям из библиотеки sklearn.

В задаче классификации модели из sklearn продемонстрировали высокую точность (Accuracy = 95.86%), а также сбалансированные значения полноты (Recall = 96.17%), точности (Precision = 95.97%) и F1-метрики (F1 Score = 95.84%). В то же время, самописная модель показала крайне низкие результаты: точность составила всего 18.29%, а значения других метрик, таких как полнота, точность и F1, также оказались крайне низкими, что указывает на ее неспособность правильно классифицировать данные. Это может быть связано с проблемой в датесете.

В задаче регрессии разрыв между моделями менее выражен, однако модели из sklearn также показали лучшее качество. Средняя абсолютная ошибка (MAE) для sklearn составила 13.13, а среднеквадратичная ошибка (MSE) — 377.79, что немного лучше результатов самописной модели (MAE = 13.18, MSE = 384.77). Это указывает на то, что самописная линейная регрессия работает корректно, но все же не достигает точности и оптимальности, предложенной готовыми библиотечными реализациями.

### Улучшение бейзлайна

#### Добавление нового признака (для задачи регрессии)

In [26]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

model = LinearRegressionCustom(lr=0.01, max_iter=2000)

model.fit(X_train_poly_scaled, y_train_new_field)

y_pred_test = model.predict(X_test_poly_scaled)

test_mae = np.mean(np.abs(y_pred_test - y_test_new_field))
test_mse = np.mean((y_pred_test - y_test_new_field) ** 2)

print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


MAE: 9.93
MSE: 247.94


### Подбор гиперпараметров и Grid Search

#### Линейная регрессия

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values.astype(np.float64)
y = data['CO2 Emissions(g/km)'].values.astype(np.float64)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid_linear = {
    'lr': [0.001, 0.01, 0.1],
    'max_iter': [1000, 2000, 5000, 10000]
}

best_params = None
best_score = float('inf')
results = []

for lr in param_grid_linear['lr']:
    for max_iter in param_grid_linear['max_iter']:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        model = LinearRegressionCustom(lr=lr, max_iter=max_iter)
        model.fit(X_train, y_train)
        
        y_pred_val = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred_val)
        
        results.append({'lr': lr, 'max_iter': max_iter, 'mse': mse})
        
        if mse < best_score:
            best_score = mse
            best_params = {'lr': lr, 'max_iter': max_iter}

best_linear_model = LinearRegressionCustom(lr=best_params['lr'], max_iter=best_params['max_iter'])
best_linear_model.fit(X_train, y_train)

y_pred_test = best_linear_model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Лучшие параметры для LinearRegressionCustom:", best_params)
print("Лучший результат на валидационной выборке (MSE):", best_score)
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Лучшие параметры для LinearRegressionCustom: {'lr': 0.1, 'max_iter': 10000}
Лучший результат на валидационной выборке (MSE): 377.9191266873952
MAE: 13.14
MSE: 377.92


In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def normalize_data(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

def encode_labels(y):
    label_to_num = {label: idx for idx, label in enumerate(np.unique(y))}
    num_to_label = {idx: label for label, idx in label_to_num.items()}
    y_encoded = np.array([label_to_num[label] for label in y])
    return y_encoded, label_to_num, num_to_label

def read_data(filepath):
    import pandas as pd
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    return X, y

def optimize_hyperparameters(train_X, train_y, test_X, test_y):
    train_X_normalized = normalize_data(train_X)
    train_y_encoded, _, _ = encode_labels(train_y)
    test_X_normalized = normalize_data(test_X)
    test_y_encoded, _, _ = encode_labels(test_y)

    param_grid = {
        'lr': [0.001, 0.01, 0.1, 0.5],
        'max_iter': [500, 1000, 2000],
        'penalty': [None, 'l1', 'l2'],
        'C': [0.1, 1.0, 10.0]
    }

    best_params = None
    best_score = 0
    results = []

    for lr in param_grid['lr']:
        for max_iter in param_grid['max_iter']:
            for penalty in param_grid['penalty']:
                for C in param_grid['C']:
                    if penalty is None and C != 1.0:
                        continue

                    model = LogisticRegressionCustom(lr=lr, max_iter=max_iter, penalty=penalty, C=C)
                    model.fit(train_X_normalized, train_y_encoded)

                    y_hat = model.predict(test_X_normalized)
                    acc = accuracy_score(test_y_encoded, y_hat)
                    f1 = f1_score(test_y_encoded, y_hat, average='weighted')

                    results.append({'lr': lr, 'max_iter': max_iter, 'penalty': penalty, 'C': C, 'accuracy': acc, 'f1': f1})

                    if f1 > best_score:
                        best_score = f1
                        best_params = {'lr': lr, 'max_iter': max_iter, 'penalty': penalty, 'C': C}

    best_model = LogisticRegressionCustom(
        lr=best_params['lr'],
        max_iter=best_params['max_iter'],
        penalty=best_params['penalty'],
        C=best_params['C']
    )
    best_model.fit(train_X_normalized, train_y_encoded)
    y_pred = best_model.predict(test_X_normalized)

    acc = accuracy_score(test_y_encoded, y_pred)
    recall = recall_score(test_y_encoded, y_pred, average='weighted')
    precision = precision_score(test_y_encoded, y_pred, average='weighted')
    f1 = f1_score(test_y_encoded, y_pred, average='weighted')

    print("Лучшие параметры:", best_params)
    print(f"Accuracy: {acc:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return best_params, results

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

best_params, results = optimize_hyperparameters(train_X, train_y, test_X, test_y)

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/

Лучшие параметры: {'lr': 0.5, 'max_iter': 500, 'penalty': 'l1', 'C': 1.0}
Accuracy: 0.1965
Recall: 0.1965
Precision: 0.0831
F1 Score: 0.1160


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации:
{'lr': 0.5, 'max_iter': 500, 'penalty': 'l1', 'C': 1.0}

In [9]:
def normalize_data(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

def encode_labels(y):
    label_to_num = {label: idx for idx, label in enumerate(np.unique(y))}
    num_to_label = {idx: label for label, idx in label_to_num.items()}
    y_encoded = np.array([label_to_num[label] for label in y])
    return y_encoded, label_to_num, num_to_label

def train_custom_logistic_regression(train_x, train_y, validation=None):
    train_x_normalized = normalize_data(train_x)
    train_y_encoded, _, _ = encode_labels(train_y)

    if validation is not None:
        validation_x_normalized = normalize_data(validation[0])
        validation_y_encoded, _, _ = encode_labels(validation[1])
    else:
        validation_x_normalized = None
        validation_y_encoded = None

    model = LogisticRegressionCustom(lr=0.5, max_iter=500, penalty='l1', C = 1.0)
    model.fit(train_x_normalized, train_y_encoded)

    y_hat = model.predict(validation_x_normalized)
    acc = metrics.accuracy_score(validation_y_encoded, y_hat)
    print(f"Accuracy = {acc}")
    recall = metrics.recall_score(validation_y_encoded, y_hat, average='weighted')
    precision = metrics.precision_score(validation_y_encoded, y_hat, average='weighted')
    f1 = metrics.f1_score(validation_y_encoded, y_hat, average='weighted')
    print(f"Recall = {recall}")
    print(f"Precision = {precision}")
    print(f"F1 Score = {f1}")

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_custom_logistic_regression(train_X, train_y, validation=(test_X, test_y))

Accuracy = 0.19579233118425518
Recall = 0.19579233118425518
Precision = 0.08286520400450481
F1 Score = 0.11559601211525375


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Задача регрессии

Лучшие параметры для регрессии:
{'lr': 0.1, 'max_iter': 10000}

In [16]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

model = LinearRegressionCustom(lr=0.01, max_iter=10000)

model.fit(X_train_poly_scaled, y_train_new_field)

y_pred_test = model.predict(X_test_poly_scaled)

test_mae = np.mean(np.abs(y_pred_test - y_test_new_field))
test_mse = np.mean((y_pred_test - y_test_new_field) ** 2)

print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


MAE: 7.99
MSE: 189.17


### Выводы

Сравнение результатов самописных моделей и моделей из библиотеки sklearn показывает значительное преимущество последних. Для задачи классификации самописная логистическая регрессия, даже после оптимизации гиперпараметров с использованием GridSearch, показала низкие результаты: Accuracy составила всего 0.1965, а F1 Score — 0.1160. В то же время логистическая регрессия из sklearn достигла гораздо более высоких значений. Это можно объяснить отсутствием каких-либо оптимизаций в самописной модели логистической регрессии или отсутствием какой-либо предобработки данных, которая есть в моделе в sklearn.

В задаче регрессии самописная линейная регрессия также отстает. После оптимизации гиперпараметров её лучшие показатели составили MAE 7.99 и MSE 189.17, тогда как линейная регрессия из sklearn с добавлением нового признака достигла результатов MAE 7.29 и MSE 139.02.

Таким образом, модели из sklearn демонстрируют значительно более высокую точность по сравнению с самописными решениями, благодаря их оптимизациям.